<a href="https://colab.research.google.com/github/akaicomet/NJOITOpenDataCenter-Pension-Record/blob/master/NJOITOpenDataCenter_Pension_Record_ModelBuilding_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install sodapy
!pip install matplotlib
!pip install hyperopt
!pip install bayesian-optimization
!pip install tbvaccine 
!pip install MulticoreTSNE
!pip install -U -q PyDrive

    100% |████████████████████████████████| 61kB 5.5MB/s 
    100% |████████████████████████████████| 829kB 13.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


  Stored in directory: /root/.cache/pip/wheels/4b/60/a2/6eec4adc73341c1a2a653022b99ff8e1cb090684dba83be683
Successfully built bayesian-optimization
  Stored in directory: /root/.cache/pip/wheels/27/59/53/3b52ee63add3692254c30d687fa4dff4d128d0557861fb028e
Successfully built MulticoreTSNE
    100% |████████████████████████████████| 993kB 19.9MB/s 


#1. Copy python files from  GItHub repository to local

In [14]:
!git clone https://github.com/akaicomet/NJOITOpenDataCenter.git
!git clone https://github.com/akaicomet/akaicomet_Utility.git
!git clone https://github.com/akaicomet/akaicomet_Chart.git
!git clone https://github.com/akaicomet/akaicomet_ML.git

Cloning into 'NJOITOpenDataCenter'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 96 (delta 54), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (96/96), done.
Cloning into 'akaicomet_Utility'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Cloning into 'akaicomet_Chart'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 58 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.
Cloning into 'akaicomet_ML'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 174 (del

#2. Run .py files in local

In [0]:
%run -i /content/akaicomet_Utility/ErrorHandling.py

%run -i /content/NJOITOpenDataCenter/YourMoneyActivePensionMembers.py 
%run -i /content/NJOITOpenDataCenter/YourMoneyRetiredPensionMembers.py
%run -i /content/NJOITOpenDataCenter/YourMoneyPensionMembersMerge.py

%run -i /content/akaicomet_Chart/X-Ray_Scan.py
%run -i /content/akaicomet_Chart/akaicomet_Chart.py

%run -i /content/akaicomet_ML/ML_Init.py
%run -i /content/akaicomet_ML/ML_Auto.py
%run -i /content/akaicomet_ML/AutoFeatureEngineering.py


#3. Import libraries

In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import random
from sodapy import Socrata
#from pandas.plotting import scatter_matrix
#from datetime import datetime
import seaborn as sb
from tqdm import tqdm
import warnings
from textwrap import wrap

In [0]:
from sklearn.externals import joblib
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#4. YourMoney and the NJOIT Open Data Center 
##4.1 YourMoney Active Pension Members API

**Pension member data load from NJOIT** 

In [16]:
client = Socrata("data.nj.gov", None)
results = client.get("45bd-gwii", limit=100000000)#100000000

**Data Cleansing and Preparation for YourMoney Active Pension Members data**

In [0]:
ActivePension_df = ActivePensionDataWrangler(results)

##4.2 YourMoney Retired Pension Members API
**Pension payment member data load from NJOIT**

In [0]:
results = client.get("8up4-62p6",   limit=100000000)

**Data Cleansing and Preparation for YourMoney Retired Pension data**

In [0]:
RetiredPension_df_grped = RetiredPensionMembersDataWrangler(results)

**Merge YourMoney Active Pension Members data and YourMoney Retired Pension data**

In [0]:
ActivePension_df = PensionMembersMerge(ActivePension_df,RetiredPension_df_grped)
del results

## 5 Recheck  Correlation Variable Validation:

In [21]:
#x_cols2 = ['_20_year_status',	'all_employers_salary_amt',		'employer_freq_pensioned', 'total_months_qty',	'location_freq_pensioned',		'current_employer_salary_rollingamt', 	'service_months_qty',	'pension_freq_pensioned', 'pension_fund_name_enc', 'pension_group_name_enc' ]
x_cols2 = ['_20_year_status','all_employers_salary_amt','employer_freq_pensioned','total_months_qty','location_freq_pensioned','service_months_qty','pension_freq_pensioned','pension_fund_name_enc','pension_group_name_enc' ]
ActivePension_df[x_cols2].corr()

,_20_year_status,all_employers_salary_amt,employer_freq_pensioned,total_months_qty,location_freq_pensioned,service_months_qty,pension_freq_pensioned,pension_fund_name_enc,pension_group_name_enc
_20_year_status,1.000000,0.295118,-0.074679,0.844411,0.082618,-0.002429,0.091064,-0.238068,-0.113550
all_employers_salary_amt,0.295118,1.000000,-0.066198,0.395345,-0.030279,-0.027708,-0.439474,0.207998,0.097105
employer_freq_pensioned,-0.074679,-0.066198,1.000000,-0.065524,0.187599,0.026186,0.040389,0.099662,-0.033238
total_months_qty,0.844411,0.395345,-0.065524,1.000000,0.067168,0.000102,0.047303,-0.187378,-0.121364
location_freq_pensioned,0.082618,-0.030279,0.187599,0.067168,1.000000,0.056022,0.308507,0.047458,-0.063150
service_months_qty,-0.002429,-0.027708,0.026186,0.000102,0.056022,1.000000,0.079330,0.052583,-0.034342
pension_freq_pensioned,0.091064,-0.439474,0.040389,0.047303,0.308507,0.079330,1.000000,-0.306339,-0.308736
pension_fund_name_enc,-0.238068,0.207998,0.099662,-0.187378,0.047458,0.052583,-0.306339,1.000000,0.185796
pension_group_name_enc,-0.113550,0.097105,-0.033238,-0.121364,-0.063150,-0.034342,-0.308736,0.185796,1.000000


# 6 Load clfs Data from Google Drive
##  6.1 Join Table 

In [0]:
#Need to change 'id' everytime

downloaded = drive.CreateFile({'id':'1XRf0Zu7tYeCLRsqxxcuT9LhfyEVDhiPo'})
downloaded.GetContentFile('clfs_score612.pkl')

downloaded = drive.CreateFile({'id':'13BdyYzlrM4eD9HpUrmAign0cmLqrY7dl'})
downloaded.GetContentFile('clfs_score623.pkl')

In [0]:
#downloaded = drive.CreateFile({'id':'1vA1Z9Y9OkTNp83XR3WLuAyA-HemgTErA'})
#downloaded.GetContentFile('clfs612.pkl')
#clfs = joblib.load('clfs612.pkl')

In [0]:
clfs_score612 = joblib.load('clfs_score612.pkl')
clfs_score623 = joblib.load('clfs_score623.pkl')

In [0]:
df_clfs_score = pd.DataFrame.from_dict(clfs_score612,orient='index')

In [0]:
df_clfs_score2 = pd.DataFrame.from_dict(clfs_score623,orient='index')

In [0]:
df_clfs_score = pd.concat([df_clfs_score, df_clfs_score2], axis=1, join='inner')

## 6.2  Additional Field and Manupulation

In [0]:
df_clfs_score['classifier'] = [i.split('|')[0] for i  in df_clfs_score.index.tolist()]

In [0]:
df_clfs_score['std'] = df_clfs_score.std(axis=1)

In [0]:
df_clfs_score.columns = ['Training', 'Test1', 'Test2', 'Test3', 'classifier','std']

In [30]:
df_clfs_score.sort_values(['classifier','Training','std'], ascending=[True, False, True])

,Training,Test1,Test2,Test3,classifier,std
AdaBoostClassifier|PCA StandardScaler|PowerTransformer|,0.949143,0.930667,0.946000,0.922,AdaBoostClassifier,0.012826
AdaBoostClassifier|PCA MinMaxScaler|PowerTransformer|,0.948857,0.938000,0.948667,0.863,AdaBoostClassifier,0.041399
AdaBoostClassifier|Raw PowerTransformer|QuantileTransformer|,0.947143,0.937333,0.945333,0.937,AdaBoostClassifier,0.005291
AdaBoostClassifier|Raw StandardScaler|PowerTransformer|,0.946286,0.932000,0.944000,0.940,AdaBoostClassifier,0.006277
AdaBoostClassifier|Raw Normalizer|StandardScaler|,0.946286,0.947333,0.948667,0.933,AdaBoostClassifier,0.007280
AdaBoostClassifier|Raw Normalizer|PowerTransformer|,0.946000,0.938000,0.945333,0.937,AdaBoostClassifier,0.004740
AdaBoostClassifier|KPCA MinMaxScaler|QuantileTransformer|,0.945429,0.945333,0.947333,0.939,AdaBoostClassifier,0.003635
AdaBoostClassifier|Raw StandardScaler|QuantileTransformer|,0.944857,0.943333,0.946000,0.934,AdaBoostClassifier,0.005475
AdaBoostClassifier|PCA PowerTransformer|QuantileTransformer|,0.944000,0.939333,0.944667,0.938,AdaBoostClassifier,0.003328
AdaBoostClassifier|PCA StandardScaler|QuantileTransformer|,0.943429,0.942000,0.944000,0.939,AdaBoostClassifier,0.002236


## 6.3 Refine Redimensioning to Validate Model 

In [32]:
manager = Manager()
clfs3 = manager.dict()
clfs_score3 = manager.dict()

flatten = lambda l: [item for sublist in l for item in sublist]

y_train = ActivePension_df.sample(n=5000, random_state=2)['IsPensionPaid'].as_matrix()
X_train = ActivePension_df[x_cols2].sample(n=5000, random_state=2)

X_train = X_train.astype(float)

scaling_def = {'MinMaxScaler': MinMaxScaler(),
               'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),
               'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False),
               'Normalizer_l2': Normalizer(norm='l2'), 'Normalizer_l1': Normalizer(norm='l1'),
               'PowerTransformer_std':PowerTransformer(standardize=True),'PowerTransformer':PowerTransformer(standardize=False)
              
              }


redemensioning_def = {'Raw':None,
                      'PCA': PCA(n_components=0.99, svd_solver='full',whiten=False),'PCAW': PCA(n_components=0.99, svd_solver='full',whiten=True),
                      'ICA_P': FastICA(n_components=len(X_train.columns), algorithm='parallel'),'ICA_D': FastICA(n_components=len(X_train.columns), algorithm='deflation'),
                      'KPCA_rbf': KernelPCA(n_components=len(X_train.columns), kernel='rbf',n_jobs=-1),'KPCA_lin': KernelPCA(n_components=len(X_train.columns), kernel='linear',n_jobs=-1),'KPCA_pol': KernelPCA(n_components=len(X_train.columns), kernel='poly',n_jobs=-1),'KPCA_sigm': KernelPCA(n_components=len(X_train.columns), kernel='sigmoid',n_jobs=-1),'KPCA_cos': KernelPCA(n_components=len(X_train.columns), kernel='cosine',n_jobs=-1),
                      'MDS_met': MDS(n_components=len(X_train.columns), metric=True, n_jobs=-1),'MDS': MDS(n_components=len(X_train.columns), metric=False, n_jobs=-1)
                     }


for clf in df_clfs_score.index.tolist():
  for i in clf.split(' ')[1:]:
    for j in i.split('|'):
      if j != '':
        scaling_def_temp = {s: scaling_def[s]  for s in scaling_def if j in s}
        redemensioning_def_temp = {i: redemensioning_def[i] for i in flatten([i.split(' ') for i in clf.split('|')])[1:] if i in redemensioning_def}
        for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def_temp), scaling(X_train,scaling_def_temp)):
          for x in data:                  
            #print(x[1])
            #print(x[0])
            p = Process(target=MLModelBuilding, args=(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3))
            #MLModelBuilding(clf.split('|')[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3)
            p.start()
            p.join(180)
            p.terminate()
              

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train score: 0.9888571428571429
Test score: 0.938
Test2 score: 0.976
RandomForestClassifier|Raw Normalizer_l2|Normalizer_l1|

Train score: 0.9448571428571428
Test score: 0.9393333333333334
Test2 score: 0.9453333333333334
GradientBoostingClassifier|Raw Normalizer_l2|Normalizer_l1|

best estimate parameters {'C': 0.988869502905378, 'class_weight': 'balanced', 'fit_intercept': True, 'intercept_scaling': 0.7033242422628925, 'loss': 'squared_hinge', 'max_iter': 16, 'tol': 0.8523805287808928}
LinearSVC|Raw Normalizer_l2|Normalizer_l1|

best estimate parameters {'C': 0.24567413803124605, 'intercept_scaling': 0.1818098692753889, 'max_iter': 5, 'solver': 'sag', 'tol': 0.9564083304485916}
LogisticRegression2|Raw Normalizer_l2|Normalizer_l1|

best estimate parameters {'learning_rate': 2.5022914363544224, 'n_estimators': 157}
Train score: 0.9405714285714286
Test score: 0.9413333333333334
Test2 score: 0.9413333333333334
AdaBoostClassifier|Raw Normalizer_l2|Normalizer_l1|



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier|PCA Normalizer_l2|Normalizer_l1|

GradientBoostingClassifier|PCA Normalizer_l2|Normalizer_l1|

best estimate parameters {'C': 0.30822292036944965, 'class_weight': 'balanced', 'fit_intercept': True, 'intercept_scaling': 0.5429067270670433, 'loss': 'squared_hinge', 'max_iter': 62, 'tol': 0.999055265579442}
LinearSVC|PCA Normalizer_l2|Normalizer_l1|

best estimate parameters {'C': 0.6368434477444558, 'intercept_scaling': 0.4004412880997891, 'max_iter': 71, 'solver': 'sag', 'tol': 0.3098946929838481}
LogisticRegression2|PCA Normalizer_l2|Normalizer_l1|

best estimate parameters {'C': 3.50225208338852, 'gamma': 99.72566131430433, 'kernel': 'rbf'}
SVC|PCA Normalizer_l2|Normalizer_l1|

best estimate parameters {'learning_rate': 1.2975880103783506, 'n_estimators': 171}
AdaBoostClassifier|PCA Normalizer_l2|Normalizer_l1|



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train score: 0.9397142857142857
Test score: 0.9406666666666667
Test2 score: 0.9406666666666667
LinearSVC|PCA StandardScalerMFST|StandardScalerMFSF|

best estimate parameters {'learning_rate': 1.0301613107599028, 'n_estimators': 237}
Train score: 0.9437142857142857
Test score: 0.9406666666666667
Test2 score: 0.9426666666666667
AdaBoostClassifier|PCA StandardScalerMTST|StandardScalerMTSF|

best estimate parameters {'learning_rate': 1.0649397761863786, 'n_estimators': 205}
Train score: 0.9448571428571428
Test score: 0.9393333333333334
Test2 score: 0.9433333333333334
AdaBoostClassifier|PCA StandardScalerMTST|StandardScalerMFST|

best estimate parameters {'learning_rate': 1.1072708161449243, 'n_estimators': 176}
Train score: 0.9437142857142857
Test score: 0.938
Test2 score: 0.9406666666666667
AdaBoostClassifier|PCA StandardScalerMTST|StandardScalerMFSF|

AdaBoostClassifier|PCA StandardScalerMTSF|StandardScalerMFST|



Process Process-71:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


KeyboardInterrupt: ignored

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/content/akaicomet_Utility/ErrorHandling.py", line 11, in wrapper
    return function(*args, **kwargs)
  File "/content/akaicomet_ML/ML_Auto.py", line 568, in MLModelBuilding
    best_params = SetHyperParamHO('AdaBoostClassifier',X_train, y_train, X_test, y_test)
  File "/content/akaicomet_Utility/ErrorHandling.py", line 11, in wrapper
    return function(*args, **kwargs)
  File "/content/akaicomet_ML/ML_Auto.py", line 78, in SetHyperParamHO
    best_params = fmin(function,param_dist,algo=tpe.suggest,max_evals=100)
  File "/usr/local/lib/python3.6/dist-packages/hyperopt/fmin.py", line 385, in fmin
    rval.exhaust()
  File "/usr/local/lib/python3.6/dist-packages/hyperopt/fmin.py", line 244, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "/usr/local/lib/python3.6/dist-packages/hyperopt/fmin.py", line 218, in run
    se

In [0]:
clfs32 = clfs3.copy()
clfs_score32 = clfs_score3.copy()

joblib.dump(clfs32, 'clfs624.pkl', compress=True)
joblib.dump(clfs_score32, 'clfs_score624.pkl', compress=True)

['clfs_score624.pkl']

In [0]:
folder_id = '1wRbAVDSYm-NJsRMUqkRsreELnGaNM88R'
file = drive.CreateFile({'mimeType': 'content/clfs624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs624.pkl')
file.Upload() 
file = drive.CreateFile({'mimeType': 'content/clfs_score624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs_score624.pkl')
file.Upload() 

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

In [0]:
#Need to change 'id' everytime

downloaded = drive.CreateFile({'id':'1MwWCrNb1IJ-6EtWKgP-1feekH43CCm5B'})
downloaded.GetContentFile('clfs624.pkl')

downloaded = drive.CreateFile({'id':'1hAxWsleDpGVZQ_0h4S0z9H8inUOJZEpl'})
downloaded.GetContentFile('clfs_score624.pkl')

In [0]:
clfs624 = joblib.load('clfs624.pkl')
clfs_score624 = joblib.load('clfs_score624.pkl')

In [0]:
df_clfs_score3 = pd.DataFrame.from_dict(clfs_score624,orient='index')

In [0]:
df_clfs_score3['classifier'] = [i.split('|')[0] for i  in df_clfs_score3.index.tolist()]

In [0]:
df_clfs_score3['std'] = df_clfs_score3.std(axis=1)

In [0]:
df_clfs_score3.columns = ['Training', 'Test1', 'Test2', 'classifier','std']

In [13]:
df_clfs_score3.sort_values(['classifier','Training','std'], ascending=[True, False, True])

,Training,Test1,Test2,classifier,std
AdaBoostClassifier|Raw MinMaxScaler|,0.939714,0.944000,0.942667,AdaBoostClassifier,0.002193
AdaBoostClassifier|Raw PowerTransformer_std|PowerTransformer|,0.939429,0.944667,0.942667,AdaBoostClassifier,0.002643
AdaBoostClassifier|Raw StandardScalerMTST|StandardScalerMTSF|StandardScalerMFST|StandardScalerMFSF|,0.939143,0.942667,0.940667,AdaBoostClassifier,0.001767
AdaBoostClassifier|PCA MinMaxScaler|,0.938286,0.944000,0.940667,AdaBoostClassifier,0.002870
AdaBoostClassifier|Raw QuantileTransformer_u|QuantileTransformer_n|,0.938286,0.944667,0.941333,AdaBoostClassifier,0.003192
AdaBoostClassifier|PCA QuantileTransformer_u|QuantileTransformer_n|,0.938000,0.944667,0.940667,AdaBoostClassifier,0.003355
AdaBoostClassifier|Raw Normalizer_l2|Normalizer_l1|,0.936286,0.948667,0.940667,AdaBoostClassifier,0.006278
AdaBoostClassifier|PCA StandardScalerMTST|StandardScalerMTSF|StandardScalerMFST|StandardScalerMFSF|,0.933429,0.938667,0.936667,AdaBoostClassifier,0.002643
AdaBoostClassifier|PCA PowerTransformer_std|PowerTransformer|,0.916571,0.910667,0.911333,AdaBoostClassifier,0.003234
AdaBoostClassifier|PCA Normalizer_l2|Normalizer_l1|,0.719143,0.670000,0.700000,AdaBoostClassifier,0.024771
